# Harvesting a series

In [ ]:
import time
import csv
import os
import math
import requests
from PIL import Image, ImageOps
from requests import ConnectionError
from recordsearch_tools.utilities import retry
from recordsearch_tools.client import RSSearchClient, RSSeriesClient
from tinydb import TinyDB, Query
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO
from IPython.display import Image as DImage
from IPython.core.display import HTML

In [ ]:
# What series do you want to harvest?
# Insert the series id between the quotes.
series = 'B6527'

In [ ]:
class SeriesHarvester():
    def __init__(self, series, control=None):
        self.series = series
        self.control = control
        self.total_pages = None
        self.pages_complete = 0
        self.client = RSSearchClient()
        self.prepare_harvest()
        self.db = TinyDB('data/db-{}.json'.format(self.series))
        self.items = self.db.table('items')
        self.images = self.db.table('images')

    def get_total(self):
        return self.client.total_results

    def prepare_harvest(self):
        if self.control:
            self.client.search(series=self.series, control=self.control)
        else:
            self.client.search(series=self.series)
        total_results = self.client.total_results
        print('{} items'.format(total_results))
        self.total_pages = math.floor(int(total_results) / self.client.results_per_page) + 1
        print(self.total_pages)

    @retry(ConnectionError, tries=20, delay=10, backoff=1)
    def start_harvest(self, page=None):
        Record = Query()
        if not page:
            page = self.pages_complete + 1
        while self.pages_complete < self.total_pages:
            if self.control:
                response = self.client.search(series=self.series, page=page, control=self.control, sort='9')
            else:
                response = self.client.search(series=self.series, page=page, sort='9')
            for result in response['results']:
                self.items.upsert(result, Record.identifier == result['identifier'])
            self.pages_complete += 1
            page += 1
            print('{} pages complete'.format(self.pages_complete))
            time.sleep(1)
        
    @retry(ConnectionError, tries=20, delay=10, backoff=1)
    def harvest_images(self):
        Record = Query()
        items = self.items.search(Record.digitised_status == True)
        headers = {'User-Agent': 'Mozilla/5.0'}
        for item in items:
            directory = os.path.join('data', 'images', '{}/{}-[{}]'.format(self.series.replace('/', '-'), item['control_symbol'].replace('/', '-').replace(' ', '-'), item['identifier']))
            if not os.path.exists(directory):
                os.makedirs(directory)
            for page in range(1, item['digitised_pages'] + 1):
                filename = '{}/{}-p{}.jpg'.format(directory, item['identifier'], page)
                print('{}, p. {}'.format(item['identifier'], page))
                if not os.path.exists(filename):
                    img_url = 'http://recordsearch.naa.gov.au/NaaMedia/ShowImage.asp?B={}&S={}&T=P'.format(item['identifier'], page)
                    response = requests.get(img_url, headers=headers, stream=True, verify=False)
                    response.raise_for_status()
                    try:
                        image = Image.open(BytesIO(response.content))
                    except IOError:
                        print('Not an image')
                    else:
                        width, height = image.size
                        image.save(filename)
                        del response
                        image_meta = {
                            'image_id': '{}-{}'.format(item['identifier'], page),
                            'identifier': item['identifier'],
                            'page': page,
                            'width': width,
                            'height': height
                        }
                        self.images.upsert(image_meta, Record.image_id == image_meta['image_id'])
                        print('Image saved')
            time.sleep(1)

In [ ]:
# Prepare the harvest
h = SeriesHarvester(series)

In [ ]:
# Start the harvest!
h.start_harvest()

In [ ]:
# Let's see how many items we've harvested
db = TinyDB('data/db-{}.json'.format(series))
items = db.table('items')
len(items)

In [ ]:
# Harvest digitised pages
h.harvest_images()

In [ ]:
display(HTML('<img src="data/images/B6527/23-SEP-1916-1-ANGLESEY-[5993849]/5993849-p1.jpg" width="200">'))